In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-jupyter.bash notebook
```

This notebook makes heavy use of the following python libraries.
- `pandas`: from the python computing stack (http://pandas.pydata.org/pandas-docs/stable/)
- `geojson`: standard JSON representation of geographic data (http://geojson.org/)
- `folium`: (https://github.com/python-visualization/folium) python bridge to leaflet (http://leafletjs.com/)

### Pick a user to work with ###

In [74]:
import emission.core.get_database as edb
import pandas as pd
all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
all_users

,user_email,uuid
0,test_aug_21,9052e86a-69b9-478a-9cba-c05a58565ab2
1,test_july_22,bb4ff90c-ea1c-4977-b4c6-55dbcd3aa591
2,test_aug_23,63c333d5-c4fa-4eec-a7ec-d088f3063ea8


In [75]:
from uuid import UUID

In [76]:
test_user_id = all_users.iloc[1].uuid # replace with UUID from above

If you want to work across multiple users, just do the same thing again

In [77]:
test_user_id_2 = all_users.iloc[2].uuid

### Preferred access technique

The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later. The timeseries is conceptually a set of streams, one for each of the types, primarily indexed by time. So you can query for all entries of a particular time within a specified time range.

In [78]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [79]:
print(test_user_id, test_user_id_2)
ts = esta.TimeSeries.get_time_series(test_user_id)
ts_2 = esta.TimeSeries.get_time_series(test_user_id_2)

bb4ff90c-ea1c-4977-b4c6-55dbcd3aa591 63c333d5-c4fa-4eec-a7ec-d088f3063ea8


#### Accessing entries directly

In [80]:
# Get all cleaned trips for the first user
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)

All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [81]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

=== Trip: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0820411, 37.3920436], "type": "Point"}
  --- Section: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0773235, 37.3946683], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-122.077133, 37.3944385], "type": "Point"} -> {"coordinates": [-122.0859763, 37.3983699], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1031161, 37.4048536], "type": "Point"} -> {"coordinates": [-122.1651563, 37.4438859], "type": "Point"}  on  MotionTypes.IN_VEHICLE
  --- Section: {"coordinates": [-122.1647388, 37.4438775], "type": "Point"} -> {"coordinates": [-122.1631988, 37.4459894], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1631156, 37.4464546], "type": "Point"} -> {"coordinates": [-122.1658293, 37.4438399], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-1

In [82]:
# Get all cleaned trips for the second user
entry_it = ts_2.find_entries(["analysis/cleaned_trip"], time_query=None)

In [83]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

=== Trip: {"coordinates": [-122.0868631, 37.390118], "type": "Point"} -> {"coordinates": [-122.0437059, 37.3210843], "type": "Point"}
=== Trip: {"coordinates": [-122.0437059, 37.3210843], "type": "Point"} -> {"coordinates": [-122.0440361, 37.3206362], "type": "Point"}
=== Trip: {"coordinates": [-122.0440361, 37.3206362], "type": "Point"} -> {"coordinates": [-122.0849814, 37.3929862], "type": "Point"}


In [84]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
entry_it_2 = ts_2.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(list(entry_it)), test_user_id_2, len(list(entry_it_2))))

From 1501545600 -> 1501632000, user bb4ff90c-ea1c-4977-b4c6-55dbcd3aa591 had 0 trips and user 63c333d5-c4fa-4eec-a7ec-d088f3063ea8 had 0 trips


#### Accessing a dataframe

In [85]:
# Get all cleaned trips for the first user
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)

In [86]:
len(ct_df)

6

In [87]:
ct_df.columns

Index(['source', 'end_ts', 'end_fmt_time', 'end_loc', 'raw_trip', 'start_ts',
       'start_fmt_time', 'start_loc', 'duration', 'distance', 'start_place',
       'end_place', 'start_local_dt_year', 'start_local_dt_month',
       'start_local_dt_day', 'start_local_dt_hour', 'start_local_dt_minute',
       'start_local_dt_second', 'start_local_dt_weekday',
       'start_local_dt_timezone', 'end_local_dt_year', 'end_local_dt_month',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_second', 'end_local_dt_weekday', 'end_local_dt_timezone',
       '_id', 'user_id', 'metadata_write_ts'],
      dtype='object')

In [88]:
ct_df[["start_loc", "end_loc", "start_fmt_time", "end_fmt_time"]]

,start_loc,end_loc,start_fmt_time,end_fmt_time
0,"{'type': 'Point', 'coordinates': [-122.0876886...","{'type': 'Point', 'coordinates': [-122.0820411...",2015-07-22T08:14:53.881000-07:00,2015-07-22T09:23:34.517000-07:00
1,"{'type': 'Point', 'coordinates': [-122.0820411...","{'type': 'Point', 'coordinates': [-122.0862974...",2015-07-22T09:25:15.494000-07:00,2015-07-22T09:30:09.011000-07:00
2,"{'type': 'Point', 'coordinates': [-122.0862974...","{'type': 'Point', 'coordinates': [-122.0801222...",2015-07-22T13:15:24.742251-07:00,2015-07-22T13:22:52.410000-07:00
3,"{'type': 'Point', 'coordinates': [-122.0801222...","{'type': 'Point', 'coordinates': [-122.1592342...",2015-07-22T13:25:45.137266-07:00,2015-07-22T13:53:13.736000-07:00
4,"{'type': 'Point', 'coordinates': [-122.1592342...","{'type': 'Point', 'coordinates': [-122.0823243...",2015-07-22T14:40:47.845961-07:00,2015-07-22T15:39:24.546000-07:00
5,"{'type': 'Point', 'coordinates': [-122.0823243...","{'type': 'Point', 'coordinates': [-122.0862601...",2015-07-22T16:00:26.295595-07:00,2015-07-22T16:28:52.997000-07:00


In [89]:
# Get all cleaned trips for the second user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
ct_df_2[["start_loc", "end_loc", "start_ts", "end_ts"]]

,start_loc,end_loc,start_ts,end_ts
0,"{'type': 'Point', 'coordinates': [-122.0868631...","{'type': 'Point', 'coordinates': [-122.0437059...",1.440348e+09,1.440349e+09
1,"{'type': 'Point', 'coordinates': [-122.0437059...","{'type': 'Point', 'coordinates': [-122.0440361...",1.440355e+09,1.440355e+09
2,"{'type': 'Point', 'coordinates': [-122.0440361...","{'type': 'Point', 'coordinates': [-122.0849814...",1.440356e+09,1.440358e+09


In [90]:
# # Get cleaned trips for the two users that started on 1st Aug UTC
# import arrow

# aug_1_tq = estt.TimeQuery("data.start_ts",
#                           arrow.get("2017-08-01").timestamp, # start of range
#                           arrow.get("2017-08-02").timestamp)  # end of range
# ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
# ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
# print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
#      (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(ct_df), test_user_id_2, len(ct_df_2)))

In [91]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [92]:
len(cs_df)

21

In [93]:
cs_df.columns

Index(['source', 'trip_id', 'start_ts', 'start_fmt_time', 'start_loc',
       'end_ts', 'end_fmt_time', 'end_loc', 'duration', 'speeds', 'distances',
       'distance', 'sensed_mode', 'end_stop', 'start_local_dt_year',
       'start_local_dt_month', 'start_local_dt_day', 'start_local_dt_hour',
       'start_local_dt_minute', 'start_local_dt_second',
       'start_local_dt_weekday', 'start_local_dt_timezone',
       'end_local_dt_year', 'end_local_dt_month', 'end_local_dt_day',
       'end_local_dt_hour', 'end_local_dt_minute', 'end_local_dt_second',
       'end_local_dt_weekday', 'end_local_dt_timezone', '_id', 'user_id',
       'metadata_write_ts', 'start_stop'],
      dtype='object')

In [94]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

,start_loc,end_loc,start_ts,end_ts,sensed_mode
0,"{'type': 'Point', 'coordinates': [-122.0876886...","{'type': 'Point', 'coordinates': [-122.0773235...",1.437578e+09,1.437578e+09,1
1,"{'type': 'Point', 'coordinates': [-122.077133,...","{'type': 'Point', 'coordinates': [-122.0859763...",1.437578e+09,1.437579e+09,2
2,"{'type': 'Point', 'coordinates': [-122.1031161...","{'type': 'Point', 'coordinates': [-122.1651563...",1.437579e+09,1.437580e+09,0
3,"{'type': 'Point', 'coordinates': [-122.1647388...","{'type': 'Point', 'coordinates': [-122.1631988...",1.437580e+09,1.437580e+09,2
4,"{'type': 'Point', 'coordinates': [-122.1631156...","{'type': 'Point', 'coordinates': [-122.1658293...",1.437580e+09,1.437581e+09,1
5,"{'type': 'Point', 'coordinates': [-122.1653719...","{'type': 'Point', 'coordinates': [-122.1559339...",1.437581e+09,1.437581e+09,2
6,"{'type': 'Point', 'coordinates': [-122.1493823...","{'type': 'Point', 'coordinates': [-122.0762984...",1.437581e+09,1.437582e+09,0
7,"{'type': 'Point', 'coordinates': [-122.0767434...","{'type': 'Point', 'coordinates': [-122.0820411...",1.437582e+09,1.437582e+09,2
8,"{'type': 'Point', 'coordinates': [-122.0820411...","{'type': 'Point', 'coordinates': [-122.0862974...",1.437582e+09,1.437583e+09,2
9,"{'type': 'Point', 'coordinates': [-122.0862974...","{'type': 'Point', 'coordinates': [-122.0801222...",1.437596e+09,1.437597e+09,1


### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [95]:
import emission.core.get_database as edb

In [96]:
edb.get_timeseries_db().find_one()

{'_id': ObjectId('60078969e84630147a20f027'),
 'user_id': UUID('9052e86a-69b9-478a-9cba-c05a58565ab2'),
 'data': {'loc': {'type': 'Point', 'coordinates': [-122.0822358, 37.3926163]},
  'fmt_time': '2015-08-21T07:54:51.095000-07:00',
  'altitude': 0,
  'ts': 1440168891.095,
  'longitude': -122.0822358,
  'filter': 'time',
  'local_dt': {'hour': 7,
   'month': 8,
   'second': 51,
   'weekday': 4,
   'year': 2015,
   'timezone': 'America/Los_Angeles',
   'day': 21,
   'minute': 54},
  'latitude': 37.3926163,
  'heading': 0,
  'sensed_speed': 0,
  'accuracy': 31.5},
 'metadata': {'write_fmt_time': '2015-08-21T07:54:51.712000-07:00',
  'write_ts': 1440168891.712,
  'time_zone': 'America/Los_Angeles',
  'platform': 'android',
  'write_local_dt': {'hour': 7,
   'month': 8,
   'second': 51,
   'weekday': 5,
   'year': 2015,
   'timezone': 'America/Los_Angeles',
   'day': 21,
   'minute': 54},
  'key': 'background/filtered_location',
  'read_ts': 0,
  'type': 'message'}}

In [97]:
edb.get_timeseries_db().distinct("metadata.key")

['background/filtered_location',
 'background/location',
 'background/motion_activity',
 'statemachine/transition',
 'stats/pipeline_time']

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [98]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

['analysis/cleaned_place',
 'analysis/cleaned_section',
 'analysis/cleaned_stop',
 'analysis/cleaned_trip',
 'analysis/cleaned_untracked',
 'analysis/confirmed_trip',
 'analysis/recreated_location',
 'analysis/smoothing',
 'segmentation/raw_place',
 'segmentation/raw_section',
 'segmentation/raw_stop',
 'segmentation/raw_trip',
 'segmentation/raw_untracked']

In [99]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id, "metadata.key": "analysis/cleaned_trip"}).count()

/Users/chuang/miniconda-4.8.3/envs/emission/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


6

In particular, you can use this to access entries that are not in the timeseries

In [100]:
edb.get_uuid_db().distinct("uuid")

[UUID('63c333d5-c4fa-4eec-a7ec-d088f3063ea8'),
 UUID('9052e86a-69b9-478a-9cba-c05a58565ab2'),
 UUID('bb4ff90c-ea1c-4977-b4c6-55dbcd3aa591')]

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [101]:
# ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [102]:
import emission.storage.decorations.timeline as esdl

In [103]:
trip_start_end_fuzz = 10 # seconds
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts - trip_start_end_fuzz, ct_df.iloc[-1].end_ts + trip_start_end_fuzz)

In [104]:
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

analysis/cleaned_trip 2015-07-22T08:14:53.881000-07:00 -> 2015-07-22T09:23:34.517000-07:00
analysis/cleaned_place 2015-07-22T09:23:34.517000-07:00 -> 2015-07-22T09:25:15.494000-07:00
analysis/cleaned_trip 2015-07-22T09:25:15.494000-07:00 -> 2015-07-22T09:30:09.011000-07:00
analysis/cleaned_place 2015-07-22T09:30:09.011000-07:00 -> 2015-07-22T13:15:24.742251-07:00
analysis/cleaned_trip 2015-07-22T13:15:24.742251-07:00 -> 2015-07-22T13:22:52.410000-07:00
analysis/cleaned_place 2015-07-22T13:22:52.410000-07:00 -> 2015-07-22T13:25:45.137266-07:00
analysis/cleaned_trip 2015-07-22T13:25:45.137266-07:00 -> 2015-07-22T13:53:13.736000-07:00
analysis/cleaned_place 2015-07-22T13:53:13.736000-07:00 -> 2015-07-22T14:40:47.845961-07:00
analysis/cleaned_trip 2015-07-22T14:40:47.845961-07:00 -> 2015-07-22T15:39:24.546000-07:00
analysis/cleaned_place 2015-07-22T15:39:24.546000-07:00 -> 2015-07-22T16:00:26.295595-07:00
analysis/cleaned_trip 2015-07-22T16:00:26.295595-07:00 -> 2015-07-22T16:28:52.997000-

In [105]:
# The timeline is an iterator, so after it is consumed, it is empty
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [106]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [107]:
for e in stl:
    print(e.metadata.key)

analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section
analysis/cleaned_stop
analysis/cleaned_section


In [108]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [109]:
for e in stl:
    print(e.metadata.key)

analysis/cleaned_section


## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.

### Plot a processed trip or set of trips ###

In [110]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [111]:
first_trip_for_user = ct_df.iloc[0]
first_trip_start_ts = first_trip_for_user.start_ts
first_trip_end_ts = first_trip_for_user.end_ts
trip_start_end_fuzz = 10 # seconds
trips_geojson_list = gfc.get_geojson_for_ts(test_user_id, first_trip_start_ts-trip_start_end_fuzz, ct_df.iloc[-1].end_ts+trip_start_end_fuzz)

In [112]:
len(trips_geojson_list)

6

In [113]:
map_list = lo.get_maps_for_geojson_trip_list(trips_geojson_list)

start_place 600789ad93f6750a7eb073f4
end_place 600789ad93f6750a7eb073f5
stop 600789ab93f6750a7eb072cb
stop 600789ab93f6750a7eb072cc
stop 600789ab93f6750a7eb072cd
stop 600789ab93f6750a7eb072ce
stop 600789ab93f6750a7eb072cf
stop 600789ab93f6750a7eb072d0
stop 600789ab93f6750a7eb072d1
section 600789aa93f6750a7eb07235
section 600789aa93f6750a7eb07244
section 600789ab93f6750a7eb07250
section 600789ab93f6750a7eb07269
section 600789ab93f6750a7eb07273
section 600789ab93f6750a7eb0729b
section 600789ab93f6750a7eb072ad
section 600789ab93f6750a7eb072ba
start_place 600789ad93f6750a7eb073f5
end_place 600789ad93f6750a7eb073f6
section 600789ab93f6750a7eb072d4
start_place 600789ad93f6750a7eb073f6
end_place 600789ad93f6750a7eb073f7
section 600789ab93f6750a7eb072e2
start_place 600789ad93f6750a7eb073f7
end_place 600789ad93f6750a7eb073f8
stop 600789ac93f6750a7eb0732c
stop 600789ac93f6750a7eb0732d
section 600789ac93f6750a7eb072f5
section 600789ac93f6750a7eb07308
section 600789ac93f6750a7eb07320
start_place 6

In [114]:
len(map_list)

6

In [115]:
map_list[0]

In [116]:
map_list[-1]

In [117]:
import branca.element as bre

In [118]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, m in enumerate(map_list[:6]):
    fig.add_subplot(nrows,ncols,i+1).add_child(m)
fig

In [119]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, map in enumerate(map_list[-6:]):
    fig.add_subplot(nrows,ncols,i+1).add_child(map)
fig

## Can you do better? ##


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [120]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

Locations go from all = 164 -> filtered = 147 -> resampled = 142


In [121]:
all_locs[["_id", "latitude", "longitude", "fmt_time"]]

,_id,latitude,longitude,fmt_time
0,60078999a12d895607014e98,37.388777,-122.087689,2015-07-22T08:14:53.881000-07:00
1,60078999a12d895607014e99,37.388777,-122.087689,2015-07-22T08:15:02.389000-07:00
2,60078999a12d895607014e9a,37.392786,-122.084974,2015-07-22T08:15:33.013000-07:00
3,60078999a12d895607014e9b,37.392570,-122.083598,2015-07-22T08:16:03.077000-07:00
4,60078999a12d895607014e9c,37.392825,-122.081985,2015-07-22T08:16:33.052000-07:00
...,...,...,...,...
159,60078999a12d895607014f37,37.392057,-122.082108,2015-07-22T09:21:34.427000-07:00
160,60078999a12d895607014f38,37.392054,-122.082148,2015-07-22T09:22:04.393000-07:00
161,60078999a12d895607014f39,37.392052,-122.082147,2015-07-22T09:22:34.407000-07:00
162,60078999a12d895607014f3a,37.392021,-122.082184,2015-07-22T09:23:04.433000-07:00


### Get the raw motion activity, in case you want to do different segmentation ###

In [122]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [123]:
all_activity.columns

Index(['confidence', 'type', 'ts', 'fmt_time', 'local_dt_hour',
       'local_dt_month', 'local_dt_second', 'local_dt_weekday',
       'local_dt_year', 'local_dt_timezone', 'local_dt_day', 'local_dt_minute',
       '_id', 'user_id', 'metadata_write_ts'],
      dtype='object')

In [124]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

Found 0 walking entries, 20 on_foot entries
Found 30 motorized entries


### Plot the location points ###

In [125]:
map_list = lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                gfc.get_feature_list_from_df(filtered_locs),
                                                gfc.get_feature_list_from_df(resampled_locs)])

Found 164 features from 164 points
Found 147 features from 147 points
Found 142 features from 142 points
Points are  [[37.3887767, -122.0876886], [37.3887767, -122.0876886], [37.3927862, -122.084974], [37.3925696, -122.0835984], [37.3928247, -122.0819854]] ...
Linestring is [[37.3887767, -122.0876886], [37.3887767, -122.0876886], [37.3927862, -122.084974], [37.3925696, -122.0835984], [37.3928247, -122.0819854]] ...
Points are  [[37.3887767, -122.0876886], [37.3927862, -122.084974], [37.3925696, -122.0835984], [37.3928247, -122.0819854], [37.3947249, -122.0816034]] ...
Linestring is [[37.3887767, -122.0876886], [37.3927862, -122.084974], [37.3925696, -122.0835984], [37.3928247, -122.0819854], [37.3947249, -122.0816034]] ...
Points are  [[37.3887767, -122.0876886], [37.39185052704932, -122.08560748990945], [37.39263585377785, -122.08401916960665], [37.392746650889414, -122.0824789053523], [37.39414185750142, -122.08172060989078]] ...
Linestring is [[37.3887767, -122.0876886], [37.3918505

In [126]:
fig = bre.Figure()
for i, map in enumerate(map_list):
    fig.add_subplot(1,3,i+1).add_child(map)
fig